In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

%load_ext autoreload
%autoreload 2

from my_library import get_pv, get_uniform, step, generate_path, Network, f, f_inv, calc_cva

In [2]:
T = 1.0
div_T = 10
path_num = 10000
learning_rate = 0.01

In [3]:
# path = np.zeros((div_T + 1, path_num, state_num))
# dw = np.random.randn(div_T, path_num)
path, dw = generate_path(T, div_T, path_num)

In [4]:
y_network = Network(input_size=8, output_size=1)
z_network = Network(input_size=8, output_size=1)

In [5]:
learning_rate = 0.001
opt_y = optim.Adam(y_network.parameters(), learning_rate)
opt_z = optim.Adam(z_network.parameters(), learning_rate)

scheduler_y = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_y, 'min', patience=100, verbose=True)
scheduler_z = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_z, 'min', patience=100, verbose=True)

for ep in range(10000):
    path, dw = generate_path(T, div_T, path_num)
    for t in range(div_T):
        if t == div_T - 1:
            targets = torch.unsqueeze(torch.FloatTensor(np.zeros(path_num)), 1)
        else:
            targets = y_network(torch.FloatTensor(path[t + 1])).detach()
            
        z = z_network(Variable(torch.FloatTensor(path[t])))
        y = y_network(torch.FloatTensor(path[t]))
        next_y = y - f(torch.unsqueeze(torch.FloatTensor(path[t]), 2), y, z)*(T/div_T) + z*torch.unsqueeze(torch.FloatTensor(dw[t]),1)

        if ep%2==0:
            critic_loss = torch.nn.SmoothL1Loss()(next_y, targets)
            opt_y.zero_grad()
            critic_loss.backward()
            opt_y.step()
        else:
            actor_loss = torch.nn.SmoothL1Loss()(next_y, targets)
            opt_z.zero_grad()
            actor_loss.backward()
            opt_z.step()

    if ep % 10 == 9:
        scheduler_y.step(critic_loss)
        scheduler_z.step(actor_loss)
        print('ep: {}'.format(ep),
              'time: {}'.format(t),
              'Training critic loss: {:.8e}'.format(critic_loss.data.numpy()),
              'Training actor loss: {:.8e}'.format(actor_loss.data.numpy()))

ep: 9 time: 9 Training critic loss: 7.33885216e-03 Training actor loss: 7.04397960e-03
ep: 19 time: 9 Training critic loss: 3.08846822e-03 Training actor loss: 2.80037662e-03
ep: 29 time: 9 Training critic loss: 2.54480448e-03 Training actor loss: 2.20473553e-03
ep: 39 time: 9 Training critic loss: 3.01643438e-03 Training actor loss: 2.70841434e-03
ep: 49 time: 9 Training critic loss: 3.60857765e-03 Training actor loss: 3.21652717e-03
ep: 59 time: 9 Training critic loss: 3.58047336e-03 Training actor loss: 3.39929666e-03
ep: 69 time: 9 Training critic loss: 3.63663095e-03 Training actor loss: 3.38047789e-03
ep: 79 time: 9 Training critic loss: 3.73187149e-03 Training actor loss: 3.46228899e-03
ep: 89 time: 9 Training critic loss: 3.74881784e-03 Training actor loss: 3.42369126e-03
ep: 99 time: 9 Training critic loss: 3.60960606e-03 Training actor loss: 3.30764172e-03
ep: 109 time: 9 Training critic loss: 3.69259343e-03 Training actor loss: 3.48648150e-03
ep: 119 time: 9 Training critic 


KeyboardInterrupt



In [5]:
learning_rate = 0.001
opt_y = optim.Adam(y_network.parameters(), learning_rate)
opt_z = optim.Adam(z_network.parameters(), learning_rate)

scheduler_y = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_y, 'min', patience=100, verbose=True)
scheduler_z = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_z, 'min', patience=100, verbose=True)

for ep in range(10000):
    path, dw = generate_path(T, div_T, path_num)
    for t in range(div_T):
        if t == div_T - 1:
            targets = torch.unsqueeze(torch.FloatTensor(np.zeros(path_num)), 1)
        else:
            targets = y_network(torch.FloatTensor(path[t + 1])).detach()
            
        z = z_network(Variable(torch.FloatTensor(path[t])))
        y = y_network(torch.FloatTensor(path[t]))
        Nume = 1.0/(1.0 - 0.5*torch.unsqueeze(torch.FloatTensor(path[t + 1][:, 2]), 1)*(T/div_T))
        next_y = Nume*(y - 0.5*(T/div_T)*(f_inv(torch.unsqueeze(torch.FloatTensor(path[t + 1]), 2)) + f_inv(torch.unsqueeze(torch.FloatTensor(path[t]), 2)) - torch.unsqueeze(torch.FloatTensor(path[t][:, 2]), 1)*y) + z*torch.unsqueeze(torch.FloatTensor(dw[t]),1))

        if ep%2==0:
            critic_loss = torch.nn.SmoothL1Loss()(next_y, targets)
            opt_y.zero_grad()
            critic_loss.backward()
            opt_y.step()
        else:
            actor_loss = torch.nn.SmoothL1Loss()(next_y, targets)
            opt_z.zero_grad()
            actor_loss.backward()
            opt_z.step()

    if ep % 10 == 9:
        scheduler_y.step(critic_loss)
        scheduler_z.step(actor_loss)
        print('ep: {}'.format(ep),
              'time: {}'.format(t),
              'Training critic loss: {:.8e}'.format(critic_loss.data.numpy()),
              'Training actor loss: {:.8e}'.format(actor_loss.data.numpy()))

ep: 9 time: 9 Training critic loss: 2.87828245e-03 Training actor loss: 2.76570627e-03
ep: 19 time: 9 Training critic loss: 2.34349119e-03 Training actor loss: 2.23977002e-03
ep: 29 time: 9 Training critic loss: 2.69611599e-03 Training actor loss: 2.47722236e-03
ep: 39 time: 9 Training critic loss: 3.04652331e-03 Training actor loss: 2.73041311e-03
ep: 49 time: 9 Training critic loss: 3.10409139e-03 Training actor loss: 2.78708409e-03
ep: 59 time: 9 Training critic loss: 3.43783619e-03 Training actor loss: 3.03619215e-03
ep: 69 time: 9 Training critic loss: 3.35663557e-03 Training actor loss: 3.07279243e-03
ep: 79 time: 9 Training critic loss: 3.40969954e-03 Training actor loss: 3.24546429e-03
ep: 89 time: 9 Training critic loss: 3.43055488e-03 Training actor loss: 3.02724773e-03
ep: 99 time: 9 Training critic loss: 3.46924178e-03 Training actor loss: 3.11545306e-03
ep: 109 time: 9 Training critic loss: 3.46240588e-03 Training actor loss: 3.11917765e-03
ep: 119 time: 9 Training critic 

In [15]:
t = 8
y = y_network(torch.FloatTensor(path[t]))
z = z_network(torch.FloatTensor(path[t]))
Nume = 1.0/(1.0 - 0.5*torch.unsqueeze(torch.FloatTensor(path[t + 1][:, 2]), 1)*(T/div_T))
y = Nume*(y - 0.5*(T/div_T)*(f_inv(torch.unsqueeze(torch.FloatTensor(path[t + 1]), 2)) + f_inv(torch.unsqueeze(torch.FloatTensor(path[t]), 2)) - torch.unsqueeze(torch.FloatTensor(path[t][:, 2]), 1)*y) + z*torch.unsqueeze(torch.FloatTensor(dw[t]),1))

In [16]:
y, torch.nn.SmoothL1Loss()(y, torch.unsqueeze(torch.FloatTensor(np.zeros(path_num)), 1))

(tensor([[-1.4119e-03],
         [ 3.5376e-04],
         [ 3.4364e-02],
         ...,
         [-1.9072e-03],
         [-1.1557e-05],
         [ 8.3567e-05]], grad_fn=<MulBackward0>),
 tensor(3.3558e-05, grad_fn=<SmoothL1LossBackward0>))

In [ ]:
y_network = Network(input_size=8, output_size=1)
z_network = Network(input_size=8, output_size=1)

opt_y = optim.Adam(y_network.parameters(), learning_rate/5.0)
opt_z = optim.Adam(z_network.parameters(), learning_rate)

for ep in range(1000):
    path, dw = generate_path(T, div_T, path_num)
    y = y_network(torch.FloatTensor(path[0]))
    targets = torch.unsqueeze(torch.FloatTensor(np.zeros(path_num)), 1)
    for t in range(div_T):
        z = z_network(torch.FloatTensor(path[t]))
        y = y - f(torch.unsqueeze(torch.FloatTensor(path[t]), 2), y, z)*(T/div_T) + z*torch.unsqueeze(torch.FloatTensor(dw[t]),1)
            
    if ep%2==0:
        critic_loss = torch.nn.SmoothL1Loss()(y, targets)
        opt_y.zero_grad()
        critic_loss.backward()
        opt_y.step()
    else:
        actor_loss = torch.nn.SmoothL1Loss()(y, targets)
        opt_z.zero_grad()
        actor_loss.backward()
        opt_z.step()
        
    if ep % 10 == 9:
        print('ep: {}'.format(ep),
              'time: {}'.format(t),
              'Training critic loss: {:.8e}'.format(critic_loss.data.numpy()),
              'Training actor loss: {:.8e}'.format(actor_loss.data.numpy()))

In [5]:
y_network = Network(input_size=8, output_size=1, hidden_size=100)
z_network = Network(input_size=8, output_size=1, hidden_size=100)

In [6]:
learning_rate = 0.01
opt_y = optim.Adam(params=y_network.parameters(), lr=learning_rate)
opt_z = optim.Adam(params=z_network.parameters(), lr=learning_rate)

scheduler_y = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_y, 'min', factor=0.5, patience=500, min_lr=1e-7, verbose=True)
scheduler_z = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_z, 'min', factor=0.5, patience=500, min_lr=1e-7, verbose=True)

for ep in range(100000):
    path, dw = generate_path(T, div_T, path_num)
    y = y_network(torch.FloatTensor(path[0]))
    targets = torch.unsqueeze(torch.FloatTensor(np.zeros(path_num)), 1)
    for t in range(div_T):
        if t == div_T - 1:
            y = y - f(torch.unsqueeze(torch.FloatTensor(path[t]), 2), y, z)*(T/div_T)
        else:
            z = z_network(torch.FloatTensor(path[t]))
            y = y - f(torch.unsqueeze(torch.FloatTensor(path[t]), 2), y, z)*(T/div_T) + z*torch.unsqueeze(torch.FloatTensor(dw[t]),1)
            
    if ep%2==0:
        critic_loss = torch.nn.SmoothL1Loss()(y, targets)
        opt_y.zero_grad()
        critic_loss.backward()
        opt_y.step()
    else:
        actor_loss = torch.nn.SmoothL1Loss()(y, targets)
        opt_z.zero_grad()
        actor_loss.backward()
        opt_z.step()
        
    if ep % 10 == 9:
        scheduler_y.step(critic_loss)
        scheduler_z.step(actor_loss)
        print('ep: {}'.format(ep),
              'time: {}'.format(t),
              'Training critic loss: {:.8e}'.format(critic_loss.data.numpy()),
              'Training actor loss: {:.8e}'.format(actor_loss.data.numpy()))

ep: 9 time: 9 Training critic loss: 1.28635131e-02 Training actor loss: 1.17101101e-02
ep: 19 time: 9 Training critic loss: 6.02662889e-03 Training actor loss: 8.95783771e-03
ep: 29 time: 9 Training critic loss: 5.72828716e-03 Training actor loss: 4.49365750e-03
ep: 39 time: 9 Training critic loss: 4.00009844e-03 Training actor loss: 3.36839864e-03
ep: 49 time: 9 Training critic loss: 2.52247415e-03 Training actor loss: 3.50779761e-03
ep: 59 time: 9 Training critic loss: 2.86813546e-03 Training actor loss: 2.25255452e-03
ep: 69 time: 9 Training critic loss: 2.63406406e-03 Training actor loss: 2.16210680e-03
ep: 79 time: 9 Training critic loss: 2.35218857e-03 Training actor loss: 2.36255443e-03
ep: 89 time: 9 Training critic loss: 2.18905578e-03 Training actor loss: 2.24175537e-03
ep: 99 time: 9 Training critic loss: 2.10595713e-03 Training actor loss: 2.17618654e-03
ep: 109 time: 9 Training critic loss: 2.09164713e-03 Training actor loss: 2.09883461e-03
ep: 119 time: 9 Training critic 


KeyboardInterrupt



In [6]:
y = y_network(torch.FloatTensor(path[0]))
targets = torch.unsqueeze(torch.FloatTensor(np.zeros(path_num)), 1)
for t in range(div_T):
    hedge = z_network(torch.FloatTensor(path[t]))
    y = y - f(torch.unsqueeze(torch.FloatTensor(path[t]), 2), y, hedge)*(T/div_T) + hedge*torch.unsqueeze(torch.FloatTensor(dw[t]),1)

In [7]:
y, torch.nn.SmoothL1Loss()(y, torch.unsqueeze(torch.FloatTensor(np.zeros(path_num)), 1))

(tensor([[-0.0097],
         [ 0.0009],
         [ 0.0218],
         ...,
         [-0.1043],
         [-0.0076],
         [-0.0020]], grad_fn=<AddBackward0>),
 tensor(0.0018, grad_fn=<SmoothL1LossBackward0>))

In [ ]:
#(1.0 - X[:, 5])*X[: ,6]*X[:, 7] - Y*X[:, 2]
ans = (1.0 - path[9][:,5])*path[9][:,6]*path[9][:,7]*0.1/(1.0 + path[9][:,2]*0.1)
ans[:12]

In [ ]:
torch.squeeze(y_network(torch.FloatTensor(path[9]))).data.numpy()[:12] - ans[:12]

In [ ]:
torch.FloatTensor((1.0 - path[9][:,5])*path[9][:,6]*path[9][:,7]/(1.0 + path[9][:,2]))

In [ ]:
y = y_network(torch.FloatTensor(path[9]))
#y = torch.unsqueeze(torch.FloatTensor((1.0 - path[9][:,5])*path[9][:,6]*path[9][:,7]/(1.0 + path[9][:,2])), 1)
next_y = y - f(torch.unsqueeze(torch.FloatTensor(path[9]), 2), y, z)*(T/div_T) #+ z*torch.unsqueeze(torch.FloatTensor(dw[t]),1)
targets = torch.unsqueeze(torch.FloatTensor(np.zeros(path_num)), 1)
torch.nn.SmoothL1Loss()(next_y, targets)

In [ ]:
next_y[:12]

In [ ]:
z_network(torch.FloatTensor(path[9])).data.numpy()[:12]

In [ ]:
cvas = np.zeros(20)
for i in range(20):
    cvas[i] = calc_cva(path[0][i], 1., 10, 10000)

In [ ]:
cvas[:12]

In [ ]:
y_network(torch.FloatTensor(path[0])).data.numpy()[:12]

In [ ]:
path[9,:,7][:12]

In [ ]:
z_network(torch.FloatTensor(path[9])).data.numpy()[:12]

In [ ]:
dw[9][:12]